# Machine learning

## Imports

In [1]:
import sys
import cufflinks
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')

sys.path.append('..')
cufflinks.go_offline()

In [2]:
from Corpus.Corpus import get_corpus, filter_binary_pn, filter_corpus_small
from auxiliar.VectorizerHelper import vectorizer, vectorizerIdf, procesar_corpus
from auxiliar import parameters
from auxiliar.HtmlParser import HtmlParser

In [3]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold
import copy

## Config

In [4]:
polarity_dim = 5
clasificadores=['lr', 'ls', 'mb', 'rf']
idf =  False
target_names=['Neg', 'Pos']
kfolds = 10
base_dir = '2-clases' if polarity_dim == 2 else ('3-clases' if polarity_dim == 3 else '5-clases')
name = 'machine_learning/tweeter/base_line'

## Get data

In [5]:
# cine = HtmlParser(200, "http://www.muchocine.net/criticas_ultimas.php", 1)
data_corpus = get_corpus('general-corpus', 'general-corpus', 1, None)

if polarity_dim == 2:
    data_corpus = filter_binary_pn(data_corpus)
#     cine = filter_binary_pn(cine.get_corpus())
elif polarity_dim == 3:
    data_corpus = filter_corpus_small(data_corpus)
#     cine = filter_corpus_small(cine.get_corpus())
# elif polarity_dim == 5:
#     cine = cine.get_corpus()
# used_data = cine[:5000]
used_data = pd.DataFrame(data_corpus)
split = used_data.shape[0] * 0.7
# data_corpus = None

#Intentando obtener datos del archivo csv...
/home/suampa/Documentos/SentimentAnalysis/Corpus/../data/general-corpus.csv
#Datos recuperados!


## Split data

In [6]:
def apply_prepro(data):
    return procesar_corpus(data, True, True, False, True)
used_data.content = used_data.content.apply(apply_prepro)

In [7]:
train_corpus = used_data.loc[:split - 1 , :]
test_corpus = used_data.loc[split:, :]

## Initialize ML

In [8]:
vect = vectorizerIdf if idf else vectorizer
ls = CalibratedClassifierCV(LinearSVC()) if polarity_dim == 2 else OneVsRestClassifier(CalibratedClassifierCV(LinearSVC()))
lr = LogisticRegression(solver='lbfgs') if polarity_dim == 2 else OneVsRestClassifier(LogisticRegression())
mb = MultinomialNB() if polarity_dim == 2 else OneVsRestClassifier(MultinomialNB())
rf = RandomForestClassifier() if polarity_dim == 2 else OneVsRestClassifier(RandomForestClassifier())

In [9]:
pipeline_ls = Pipeline([
    ('vect', copy.deepcopy(vect)),
    ('ls', ls)
])
pipeline_lr = Pipeline([
    ('vect', copy.deepcopy(vect)),
    ('lr', lr)
])
pipeline_mb = Pipeline([
    ('vect', copy.deepcopy(vect)),
    ('mb', mb)
])
pipeline_rf = Pipeline([
    ('vect', copy.deepcopy(vect)),
    ('rf', rf)
])

In [10]:
pipelines = {
    'ls': pipeline_ls,
    'lr': pipeline_lr,
    'mb': pipeline_mb,
    'rf': pipeline_rf
}
pipelines_train = {
    'ls': ls,
    'lr': lr,
    'mb': mb,
    'rf': rf
}

## Train

In [11]:
folds = pd.read_pickle('folds.pkl') # k-folds precargados
folds = folds.values

In [12]:
x_vect = vect.fit_transform(train_corpus.content, train_corpus.polarity).toarray()

In [13]:
vect.vocabulary_

{'gracias': 4463,
 'mar': 5872,
 'off': 6611,
 'pensando': 7012,
 'regalito': 8019,
 'sinde': 8709,
 'va': 9534,
 'sgae': 8642,
 'cuando': 2376,
 'van': 9576,
 'sus': 8966,
 'corruptos': 2259,
 'intento': 5113,
 'no': 6471,
 'sacar': 8395,
 'conclusiones': 1960,
 'conozco': 2037,
 'adicto': 204,
 'drama': 3135,
 'ja': 5243,
 'suena': 8903,
 'algo': 374,
 'toca': 9210,
 'grabación': 4457,
 'especial': 3647,
 'navideño': 6395,
 'mari': 5896,
 'crismas': 2335,
 'buen': 1230,
 'día': 3190,
 'primero': 7502,
 'mandar': 5821,
 'abrazo': 42,
 'grande': 4473,
 'miguel': 6090,
 'su': 8869,
 'familia': 3958,
 'hoy': 4762,
 'podría': 7238,
 'ser': 8595,
 'grandeza': 4475,
 'humana': 4774,
 'escaño': 3580,
 'listo': 5576,
 'empezar': 3362,
 'congreso': 2014,
 'buenos': 1246,
 'días': 3192,
 'em': 3320,
 'ira': 5186,
 'puente': 7679,
 'si': 8651,
 'vais': 9545,
 'dejeis': 2643,
 'llevar': 5631,
 'tableta': 9002,
 'pc': 6946,
 'luego': 5688,
 'orbyt': 6690,
 'momento': 6200,
 'digo': 2961,
 'más': 6

In [14]:
results = {}
with tqdm(total=len(clasificadores) * 10) as pbar:
    for c in clasificadores:
        results[c] = { 'real': {}, 'predicted': {} }
        i = 0
        for train_index, test_index in folds:
            train_x = x_vect[train_index]
            train_y = train_corpus.polarity[train_index]
            test_x = x_vect[test_index]
            test_y = train_corpus.polarity[test_index]

            pipelines_train[c].fit(train_x, train_y)

            predicted = pipelines_train[c].predict(test_x)
            
            results[c]['real'][i] = test_y.values.tolist()
            results[c]['predicted'][i] = predicted.tolist()
            i = i + 1

            pbar.update(1)

    

100%|██████████| 40/40 [10:32<00:00, 37.29s/it]


In [15]:
pd.DataFrame(results).to_pickle('results/'+name+'/'+base_dir+'/results.pkl')